# Radial distortion measurement

02.06.2025 - Dominique Humbert
Initial version: heavily inspired by Discorpy's documentation (https://discorpy.readthedocs.io)

Make sure the preferred method and the other give the same values (there is a mistake somewhere)

## User manual

### Dot detection mode
1. Fill in the inputs
2. Image a grid of dots. The more the better for the fitting. ISO 9030, ISO 17850 calls for at least 15 dots in height or width of the image.
3. Image a line of dots with one in the center of the FoV for the measurement. ISO 9030, ISO 17850 calls for at least 15 dots in height or width of the image.
4. Run the code
 
 ### Dot prediction mode

Preferred method

1. Fill in the inputs
2. Image a grid of dots. The more the better for the fitting. ISO 9030, ISO 17850 calls for at least 15 dots in height or width of the image.
3. Input theoretical coordinate into the forward model to get the distorted position and compute the distortion


## Input parameters



In [ ]:
calibration_path = r"measurments/dot.jpg"
distortion_measurment_path = r"measurments/dot1.jpg"    # image patht thi the line of dots
outputs_path = r"outputs"                               # Output folder
poly_coef = 5                                           # Number of polynomial coefficients
image_fov_y = 2056   # Complete FoV in px
image_fov_x = 2464   # Complete FoV in px


INVERT = True   # True if black dots on white bcg

PREFERRED_METHOD = False
preferred_method_dots = 10  # N points at which to compute the distotions




## Computing the radial distortion coefficients

In [ ]:
import numpy as np
import discorpy.util.utility as utils
import discorpy.losa.loadersaver as losa
import discorpy.prep.preprocessing as prep
import discorpy.proc.processing as proc
import discorpy.post.postprocessing as post

mat0 = losa.load_image(calibration_path) # Load image
(height, width) = mat0.shape

mat1 = prep.binarization(mat0)
# Calculate the median dot size and distance between them.
(dot_size, dot_dist) = prep.calc_size_distance(mat1)
# Remove non-dot objects
mat1 = prep.select_dots_based_size(mat1, dot_size)
# Remove non-elliptical objects
mat1 = prep.select_dots_based_ratio(mat1)
losa.save_image(outputs_path + "/segmented_dots.jpg", mat1) # Save image for checking
# Calculate the slopes of horizontal lines and vertical lines.
hor_slope = prep.calc_hor_slope(mat1)
ver_slope = prep.calc_ver_slope(mat1)

# Group points to horizontal lines
list_hor_lines = prep.group_dots_hor_lines(mat1, hor_slope, dot_dist)
list_ver_lines = prep.group_dots_ver_lines(mat1, ver_slope, dot_dist)
list_hor_lines = prep.remove_residual_dots_hor(list_hor_lines, hor_slope)
list_ver_lines = prep.remove_residual_dots_ver(list_ver_lines, ver_slope)
# Save output for checking
losa.save_plot_image(outputs_path + "/horizontal_lines.png", list_hor_lines, height, width)
losa.save_plot_image(outputs_path + "/vertical_lines.png", list_ver_lines, height, width)


list_hor_data = post.calc_residual_hor(list_hor_lines, 0.0, 0.0)
list_ver_data = post.calc_residual_ver(list_ver_lines, 0.0, 0.0)
losa.save_residual_plot(outputs_path + "/hor_residual_before_correction.png",
                      list_hor_data, height, width)
losa.save_residual_plot(outputs_path + "/ver_residual_before_correction.png",
                      list_ver_data, height, width)

# Calculate the center of distortion
(xcenter, ycenter) = proc.find_cod_coarse(list_hor_lines, list_ver_lines)
# Calculate coefficients of the correction model
list_fact_bw = proc.calc_coef_backward(list_hor_lines, list_ver_lines,
                                    xcenter, ycenter, poly_coef)
# Save the results for later use.
losa.save_metadata_txt(outputs_path + "/coefficients_radial_distortion_backward.txt",
                     xcenter, ycenter, list_fact_bw)


# Calculate coefficients of the correction model
list_fact_fw = proc.calc_coef_forward(list_hor_lines, list_ver_lines,
                                    xcenter, ycenter, poly_coef)
losa.save_metadata_txt(outputs_path + "/coefficients_radial_distortion_forward.txt",
                     xcenter, ycenter, list_fact_fw)

# Load coefficients from previous calculation if need to
# (xcenter, ycenter, list_fact) = losa.load_metadata_txt(
#     outputs_path + "/coefficients_radial_distortion.txt")



## Correct the image

In [ ]:
mat1 = losa.load_image(distortion_measurment_path) # Load raw image
corrected_mat = post.unwarp_image_backward(mat1.astype(np.uint8), xcenter, ycenter, list_fact_bw)
losa.save_image(outputs_path + "/corrected_image.png", corrected_mat)
losa.save_image(outputs_path + "/difference.png", corrected_mat - mat1)

# # Decorrect the image
# mat2 = losa.load_image('outputs/corrected_image.png') # Load corrected image
# decorrected_mat = post.unwarp_image_backward(mat2.astype(np.uint8), xcenter, ycenter, list_fact_fw)
# losa.save_image(outputs_path + "/decorrected_image.png", decorrected_mat)
# losa.save_image(outputs_path + "/decor_difference.png", decorrected_mat - mat1)

## Dot detection mode

### Finding the center of the markers in the corrected and raw FoV

In [ ]:
if not PREFERRED_METHOD:
    import cv2
    import numpy as np
    import matplotlib.pyplot as plt
    
    marker_halfsize = 20
    # Load image (grayscale or binary)
    img = cv2.imread(outputs_path + "/corrected_image.png", cv2.IMREAD_GRAYSCALE)
    # Threshold the image (you can tweak threshold value or use adaptive/otsu)
    _, binary = cv2.threshold(img, 70, 255, cv2.THRESH_BINARY)
    
    # Invert if needed (dots should be white on black background)
    if INVERT:
        binary = cv2.bitwise_not(binary)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    centroids_c = []
    # Loop through each detected contour (dot)
    for cnt in contours:
        M = cv2.moments(cnt)
        if M["m00"] != 0:
            cx = M["m10"] / M["m00"]
            cy = M["m01"] / M["m00"]
            centroids_c.append((cx, cy))
    centroids_c = np.array(centroids_c)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    output = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    i = 0
    for (x, y) in centroids_c:
        cv2.rectangle(output,(int(x-marker_halfsize),int(y-marker_halfsize)),(int(x+marker_halfsize),int(y+marker_halfsize)),(0,255,0),3)
        cv2.putText(output,str(i),(int(x),int(y)), font, 2,(0,255,0),4,cv2.LINE_AA)
        i+=1
    cv2.imwrite(outputs_path+'/annoted_corrected_image.png', output)
    # cv2.namedWindow('Centroids markers', cv2.WINDOW_KEEPRATIO)
    # cv2.imshow("Centroids markers", output)
    # cv2.resizeWindow('Centroids markers', 400, 400)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    
    # Do the same or the raw image
    img = cv2.imread(distortion_measurment_path, cv2.IMREAD_GRAYSCALE)
    # Threshold the image (you can tweak threshold value or use adaptive/otsu)
    _, binary = cv2.threshold(img, 70, 255, cv2.THRESH_BINARY)
    # Invert if needed (dots should be white on black background)
    if INVERT:
        binary = cv2.bitwise_not(binary)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    centroids_r = []
    # Loop through each detected contour (dot)
    for cnt in contours:
        M = cv2.moments(cnt)
        if M["m00"] != 0:
            cx = M["m10"] / M["m00"]
            cy = M["m01"] / M["m00"]
            centroids_r.append((cx, cy))
    centroids_r = np.array(centroids_r)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    output = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    i = 0
    for (x, y) in centroids_r:
        cv2.rectangle(output,(int(x-marker_halfsize),int(y-marker_halfsize)),(int(x+marker_halfsize),int(y+marker_halfsize)),(0,255,0),3)
        cv2.putText(output,str(i),(int(x),int(y)), font, 2,(0,255,0),4,cv2.LINE_AA)
        i+=1
    cv2.imwrite(outputs_path+'/annoted_raw_image.png', output)
    # cv2.namedWindow('Centroids markers', cv2.WINDOW_KEEPRATIO)
    # cv2.imshow("Centroids markers", output)
    # cv2.resizeWindow('Centroids markers', 400, 400)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

### Remove some points if needed

In [ ]:
if not PREFERRED_METHOD:
    print(centroids_r.shape)
    centroids_r = np.delete(centroids_r,0,axis =0)
    print(centroids_r.shape)


### Computing the distortion

In [ ]:
if not PREFERRED_METHOD:
      import pandas as pd
      
      if centroids_c.shape != centroids_r.shape:
            raise ValueError("Not all centroids were found in one of the image. Remove the correct one.")
      
      h = ((centroids_r[:,0]-image_fov_x/2)**2 + (centroids_r[:,1]-image_fov_y/2)**2)**.5
      h0 = ((centroids_c[:,0]-image_fov_x/2)**2 + (centroids_c[:,1]-image_fov_y/2)**2)**.5
      distortion = 100*(h-h0)/h0

      ###########################################
      middle_point = distortion.shape[0]//2-1
      ###########################################

      fov = np.linspace(0,1,distortion.shape[0]-middle_point)
      formatted_data = pd.DataFrame({'Normalized FoV': fov,'distortion [%]': distortion[middle_point:]})

## Dot prediction mode


### Compute the distorted position and relative distortion

In [ ]:
if PREFERRED_METHOD:
    import pandas as pd
    (xcenter, ycenter, list_fact) = losa.load_metadata_txt(outputs_path + "/coefficients_radial_distortion_forward.txt")
    center_delta = ((xcenter-image_fov_x/2)**2 + (ycenter-image_fov_y/2)**2)**.5
    radial_fov = (image_fov_x**2+image_fov_y**2)**.5
    print('Radial difference between the sidtortion center and Fov center:',center_delta,'px (',100*center_delta/(radial_fov/2),'%)')
    distortion = []
    for i in range(preferred_method_dots):
        x = np.linspace(image_fov_x/2,image_fov_x,preferred_method_dots)
        y = np.linspace(image_fov_y/2,image_fov_y,preferred_method_dots)
        [xd, yd] = utils.find_point_to_point((x[i],y[i]), xcenter, ycenter, list_fact_fw, output_order='xy')

        h = (xd**2 + yd**2)**.5
        h0 = (x[i]**2 + y[i]**2)**.5
        distortion.append(100*(h-h0)/h0)

    distortion = np.array(distortion)
    fov = np.linspace(0,1,preferred_method_dots)
    formatted_data = pd.DataFrame({'Normalized FoV': fov,'distortion [%]': distortion})

## Output

In [ ]:
print('Fov 0 should have close to 0% distortion. If not, check the middle point computation or make sure the distortion and fov centers are meant to not be confounded')
print(formatted_data)
formatted_data.to_csv(outputs_path+'/distortion.txt', index=False, sep=',', header=True)